# 06_Model_Interpretability_SHAP
Compute SHAP values for a trained tree model.

In [ ]:
# Common imports for the project
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
%matplotlib inline


In [ ]:
import joblib
import shap
from src.preprocessing import basic_cleaning
from src.feature_engineering import create_features

In [ ]:
model_path = '../models/lgbm.joblib'
if not os.path.exists(model_path):
    model_path = '../models/xgb.joblib' if os.path.exists('../models/xgb.joblib') else '../models/rf.joblib'
print('Using model:', model_path)
model = joblib.load(model_path)
df = pd.read_csv('../data/merged_data.csv')
df = basic_cleaning(df)
df = create_features(df)
target = 'default'
exclude = [target, 'id', 'index', 'source']
X = df[[c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]].fillna(0)
X_sample = X.sample(n=min(1000, len(X)), random_state=42)

In [ ]:
try:
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)
    shap.summary_plot(shap_values, X_sample, show=True)
except Exception as e:
    print('TreeExplainer failed:', e)
    explainer = shap.KernelExplainer(lambda x: model.predict_proba(x)[:,1], X_sample.iloc[:100])
    shap_values = explainer.shap_values(X_sample)
    shap.summary_plot(shap_values, X_sample, show=True)